In [23]:
import os
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceTGIGenerator
import pyodbc

from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import pandas as pd

from transformers import BloomForCausalLM
from transformers import BloomForTokenClassification
from transformers import BloomForTokenClassification
from transformers import BloomTokenizerFast
import torch
from transformers import pipeline
import tensorflow as tf
import keras

In [19]:
import pyodbc
#Add your own SQL Server IP address, PORT, UID, PWD and Database
conn = pyodbc.connect(
    'DRIVER={PostgreSQL Unicode};SERVER=localhost;PORT=5432;DATABASE=postgres;UID=postgres;PWD=mysecretpassword', autocommit=True)
cur = conn.cursor()

# Update rows with empty attributes in Vulnerabilities table
cur.execute("UPDATE Vulnerabilities SET vulnerability_id = 'None' WHERE vulnerability_id = ''")
cur.execute("UPDATE Vulnerabilities SET description = 'None' WHERE description = ''")
cur.execute("UPDATE Vulnerabilities SET severity = 'None' WHERE severity = ''")
cur.execute("UPDATE Vulnerabilities SET required_action = 'None' WHERE required_action = ''")

# Update rows with empty attributes in AffectedProducts table
cur.execute("UPDATE AffectedProducts SET vulnerability_id = 'None' WHERE vulnerability_id = ''")
cur.execute("UPDATE AffectedProducts SET product_name = 'None' WHERE product_name = ''")
cur.execute("UPDATE AffectedProducts SET version = 'None' WHERE version = ''")

# Update rows with empty attributes in ReferenceData table
cur.execute("UPDATE ReferenceData SET vulnerability_id = 'None' WHERE vulnerability_id = ''")
cur.execute("UPDATE ReferenceData SET url = 'None' WHERE url = ''")
cur.execute("UPDATE ReferenceData SET description = 'None' WHERE description = ''")

cur.execute("ALTER TABLE Vulnerabilities ALTER COLUMN published_date TYPE TEXT")
#cur.execute("UPDATE Vulnerabilities SET published_date = TO_CHAR(published_date, 'YYYY-MM-DD') WHERE published_date IS NOT NULL")


In [20]:
import pyodbc
#Add your own SQL Server IP address, PORT, UID, PWD and Database
conn = pyodbc.connect(
    'DRIVER={PostgreSQL Unicode};SERVER=localhost;PORT=5432;DATABASE=postgres;UID=postgres;PWD=mysecretpassword', autocommit=True)
cur = conn.cursor()

cur.execute("SELECT * FROM Vulnerabilities")
rows = cur.fetchall()

In [28]:


# Set the environment variable OPENAI_API_KEY
os.environ['OPENAI_API_KEY'] = "Your API Key here"

# Write documents to InMemoryDocumentStore
document_store = InMemoryDocumentStore()
for row in rows:
    document_store.write_documents([
        Document(content=str(row)), 
    ])

# Build a RAG pipeline
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator()

rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
#rag_pipeline.add_component("llm", HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-v0.1"))
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

# Ask a question
question = "Give me a list of 10 CVEs that would impact Apple iOS devices ranked by severity, and explain why they are ranked that way?"
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

print(results["llm"]["replies"])

Ranking by BM25...:   0%|          | 0/1084 [00:00<?, ? docs/s]

['1. CVE-2023-28206 - This CVE allows an app to execute code with kernel privileges, which is a severe security risk as it can lead to complete control of the device.\n\n2. CVE-2023-41992 - This CVE allows for local privilege escalation on Apple iOS devices, which can lead to unauthorized access to sensitive information and control of the device.\n\n3. CVE-2023-41990 - This CVE allows for code execution when processing a font file, which can be exploited by attackers to run malicious code on the device.\n\n4. CVE-2023-41991 - This CVE allows a malicious app to bypass signature validation, potentially leading to the installation of unauthorized or malicious apps on the device.\n\n5. CVE-2023-37450 - This CVE allows an attacker to execute code when processing web content, which can lead to unauthorized access to the device and data theft.\n\n6. CVE-2023-28205 - This CVE leads to code execution when processing maliciously crafted web content, posing a risk of unauthorized access and data 